In [1]:
# importacao de bibliotecas
import pandas as pd
from unidecode import unidecode
import numpy as np
from sqlalchemy import create_engine

%load_ext sql

In [2]:
# Buscar dados dos arquivos do excel

df_excel_total = pd.read_excel(r'G:\OneDrive - Default Directory\OneDrive - rede.sp\Arquivos_CCIH\ARQUIVOS_ORIGINAIS\Relatorio_CCIH_01_07_ate_19_08_2024.xlsx')

In [3]:


columns_to_drop = ['Unnamed: 0']

df_excel_total = df_excel_total.drop(columns_to_drop,errors='ignore', axis=1)
df_excel_total.shape[0]

510

In [4]:
#Apresentacao dos dados

df_excel_total.head()

,parametroDataInicial,parametroDataFinal,ParametroUsuarioLogado,DataSistema,Clinica,NomeClinica,NumeroPedido,NomePaciente,DataNascimento,Nome,Resultado
0,2024-07-01,2024-08-19,HENRIQLUSTOSA,2024-07-03,6URO,UROLOGIA_6º AND - AMB,10836216,ADALBERTO CORREA,1948-06-14,UROCULTURA,Isolado [1]: Klebsiella pneumoniae ssp pneumoniae
1,2024-07-01,2024-08-19,HENRIQLUSTOSA,2024-08-07,6URO,UROLOGIA_6º AND - AMB,10852497,ADALBERTO CORREA,1948-06-14,UROCULTURA,Isolado [1]: Klebsiella pneumoniae
2,2024-07-01,2024-08-19,HENRIQLUSTOSA,2024-07-25,6URO,UROLOGIA_6º AND - AMB,10845429,ADILSON ANTUNES BITENCOURT,1947-01-21,UROCULTURA,Isolado [1]: Escherichia coli
3,2024-07-01,2024-08-19,HENRIQLUSTOSA,2024-07-25,6URO,UROLOGIA_6º AND - AMB,10845429,ADILSON ANTUNES BITENCOURT,1947-01-21,UROCULTURA,Isolado [2]: Enterococcus faecalis
4,2024-07-01,2024-08-19,HENRIQLUSTOSA,2024-08-14,6URO,UROLOGIA_6º AND - AMB,10855366,AFONSO DA CONCEIÇÃO FELIX,1947-04-11,UROCULTURA,Isolado [1]: Proteus mirabilis


In [5]:
# Busca dos resultados de exames positivados de MDR de com duas linhas no dataframe original
df_excel = df_excel_total
df_resultados_linha_2 = df_excel.loc[((df_excel['Resultado'] == 'Enterococcus resistente a glicopeptideos (VRE), consultar CCIH.') | (df_excel['Resultado'] == 'Cepa resistente a carbapenemicos, consultar CCIH.')| (df_excel['Resultado'] == 'Enterococcus Vanco Resistente (VRE), consultar CCIH.')| (df_excel['Resultado'] == 'Ampicilina - Resistente')) & (df_excel['NomePaciente'] == df_excel['NomePaciente'].shift(1))]

df_resultados_linha_1 = df_excel.loc[(df_resultados_linha_2.index - 1)]

# Alteração do índice no dataframe que buscou os dados do excel para substituição da linha 2 para a linhas 1
df_resultados_linha_2.index = df_resultados_linha_2.index - 1

df_resultados_linha_1['ComplementoResultado'] = df_resultados_linha_2.Resultado.values

df_resultados_linha_1.shape[0]

32

In [6]:
# Busca dos resultados de exames positivados de MDR de com apenas 1 linha no dataframe original
df_excel_2 = df_excel_total

df_resultados_apenas_linha_1 = df_excel_2.loc[((df_excel_2['Resultado'] == 'Enterococcus resistente a glicopeptideos (VRE), consultar CCIH.') | (df_excel_2['Resultado'] == 'Cepa resistente a carbapenemicos, consultar CCIH.')| (df_excel_2['Resultado'] == 'Enterococcus Vanco Resistente (VRE), consultar CCIH.')| (df_excel_2['Resultado'] == 'Ampicilina - Resistente')) & (df_excel_2['NomePaciente'] != df_excel_2['NomePaciente'].shift(1))].copy()

df_resultados_apenas_linha_1['ComplementoResultado'] = df_resultados_apenas_linha_1['Resultado']

df_resultados_apenas_linha_1.loc[:, 'Resultado'] = np.nan

df_resultados_apenas_linha_1.shape[0]

0

In [7]:
# Dataframe com todos os resultados de MDR positivados

df_resultados_excel = pd.concat([df_resultados_linha_1, df_resultados_apenas_linha_1], ignore_index=True)

In [8]:
## Conexão com a base de dados postgree

import pyodbc as podbc

cnxn = podbc.connect("Driver={SQL Server Native Client 11.0};Server=hspmins18;Database=Isolamento_Versao_2;uid=hspmApp;pwd=SoundG@rden=1")

In [9]:
## Acesso a base de dados

sql = "SELECT * FROM [Isolamento_Versao_2].[dbo].[CCIH] WHERE    [DataSistema] BETWEEN '2024-01-07' AND '2024-19-08' "
df_resultados_sql = pd.read_sql(sql,cnxn)
df_resultados_sql.shape[0]

<ipython-input-9-1bf3fd8e1700>:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_resultados_sql = pd.read_sql(sql,cnxn)


12

In [10]:
# from excel
# df_resultados_excel

# from data base 
# df_resultados_sql

df_resultados_inner_join = pd.merge(df_resultados_excel, df_resultados_sql, on=['NumeroPedido', 'Resultado'], how='inner')
df_resultados_inner_join.shape[0]


12

In [11]:
# Perform a left join
df_merge_sql = pd.merge(df_resultados_sql, df_resultados_excel, on=['NumeroPedido', 'Resultado'], how='left', indicator=True)
df_merge_sql.shape[0]

12

In [12]:
# Finding columns that contain '__y'
columns_to_drop_sql = [col for col in df_merge_sql.columns if '_y' in col]
# Adding another column to drop
additional_columns_sql = ['_merge'] 
# Append or extend the list with new column na
columns_to_drop_sql.extend(additional_columns_sql)


df_anti_join_sql= df_merge_sql[df_merge_sql['_merge'] == 'left_only'].drop(columns=columns_to_drop_sql)
df_anti_join_sql.shape[0]

0

In [13]:
# Perform a left join
df_merge_excel = pd.merge(df_resultados_excel, df_resultados_sql, on=['NumeroPedido', 'Resultado'], how='left', indicator=True)
# Finding columns that contain '__y'
columns_to_drop_excel = [col for col in df_merge_sql.columns if '_y' in col]

# List of additional columns to drop
additional_columns_excel = ['_merge','posicao','Prontuario','id'] 
# Append or extend the list with new column names
columns_to_drop_excel.extend(additional_columns_excel) 
df_anti_join_excel = df_merge_excel[df_merge_excel['_merge'] == 'left_only'].drop(columns=columns_to_drop_excel)
df_anti_join_excel.shape[0]

20

In [14]:
cnxn = podbc.connect("Driver={SQL Server Native Client 11.0};Server=hspmins18;Database=Isolamento_Versao_2;uid=hspmApp;pwd=SoundG@rden=1")
cursor = cnxn.cursor()
try:
    for index, row in df_resultados_inner_join.iterrows():
        # Prepare the SQL query
        sql = """
        UPDATE [Isolamento_Versao_2].[dbo].[CCIH]
        SET [ComplementoResultado] = ?
        WHERE [NumeroPedido] = ? and [Resultado] = ?;
        """
        # Execute the query with parameters
        cursor.execute(sql, (row['ComplementoResultado_x'], row['NumeroPedido'] ,row['Resultado']))
         
    # Commit all updates
    cnxn.commit()
    print("All updates were successful.")
except Exception as e:
    print("Error occurred:", e)
    cnxn.rollback()  # Roll back in case of any error
finally:
    cursor.close()
    cnxn.close()

All updates were successful.


In [15]:
df_anti_join_excel.rename(columns=lambda x: x.replace('_x', ''), inplace=True)
df_anti_join_excel

,parametroDataInicial,parametroDataFinal,ParametroUsuarioLogado,DataSistema,Clinica,NomeClinica,NumeroPedido,NomePaciente,DataNascimento,Nome,Resultado,ComplementoResultado,Ativo
0,2024-07-01,2024-08-19,HENRIQLUSTOSA,2024-07-29,2GER,GERIATRIA_2º AND - AMB,10823657,ARACY RHORMENS,1935-06-18,UROCULTURA,Isolado [1]: Klebsiella pneumoniae,"Cepa resistente a carbapenemicos, consultar CCIH.",NaN
4,2024-07-01,2024-08-19,HENRIQLUSTOSA,2024-08-03,1UT6,UTI_6º AND,10846896,CELINA PEREIRA CORDEIRO,1945-11-24,CULTURA GERAL,Isolado [1]: Acinetobacter baumannii complex,"Cepa resistente a carbapenemicos, consultar CCIH.",NaN
5,2024-07-01,2024-08-19,HENRIQLUSTOSA,2024-08-06,9C,9º AND_C ENFERMARIA GERAL,10847670,CELINA PEREIRA CORDEIRO,1945-11-24,UROCULTURA,Isolado [1]: Klebsiella pneumoniae,"Cepa resistente a carbapenemicos, consultar CCIH.",NaN
7,2024-07-01,2024-08-19,HENRIQLUSTOSA,2024-08-12,1PS,PRONTO SOCORRO ADULTO_1º AND,10839354,CLEA APARECIDA DE SOUZA,1954-04-09,CULTURA GERAL,Isolado [1]: Klebsiella pneumoniae,"Cepa resistente a carbapenemicos, consultar CCIH.",NaN
8,2024-07-01,2024-08-19,HENRIQLUSTOSA,2024-08-12,1PS,PRONTO SOCORRO ADULTO_1º AND,10839354,CLEA APARECIDA DE SOUZA,1954-04-09,CULTURA GERAL,Isolado [2]: Enterococcus faecium,Enterococcus resistente a glicopeptideos (VRE)...,NaN
10,2024-07-01,2024-08-19,HENRIQLUSTOSA,2024-07-31,10A,10º AND_ UTI,10839211,DIONYSIO CAETANO,1939-07-14,HEMOCULTURA 2º AMOSTRA,Isolado [1]: Pseudomonas aeruginosa,"Cepa resistente a carbapenemicos, consultar CCIH.",NaN
11,2024-07-01,2024-08-19,HENRIQLUSTOSA,2024-08-11,12A,12º AND_ - ENF,10849938,EDIVAR GRACIANO DA SILVA,1945-03-03,CULTURA GERAL,Isolado [1]: Enterococcus faecium,Enterococcus resistente a glicopeptideos (VRE)...,NaN
15,2024-07-01,2024-08-19,HENRIQLUSTOSA,2024-07-25,1PS,PRONTO SOCORRO ADULTO_1º AND,10844180,EVELINE APARECIDA DE ALMEIDA CARLOS,1983-10-12,CULTURA GERAL,Isolado [1]: Klebsiella pneumoniae,"Cepa resistente a carbapenemicos, consultar CCIH.",NaN
16,2024-07-01,2024-08-19,HENRIQLUSTOSA,2024-08-09,10A,10º AND_ UTI,10824488,LENI DA SILVA,1946-08-06,HEMOCULTURA 1º AMOSTRA,Isolado [1]: Acinetobacter baumannii,"Cepa resistente a carbapenemicos, consultar CCIH.",NaN
17,2024-07-01,2024-08-19,HENRIQLUSTOSA,2024-08-05,10A,10º AND_ UTI,10839219,LENI DA SILVA,1946-08-06,HEMOCULTURA 1º AMOSTRA,Isolado [2]: Acinetobacter baumannii,"Cepa resistente a carbapenemicos, consultar CCIH.",NaN


In [16]:
cnxn = podbc.connect("Driver={SQL Server Native Client 11.0};Server=hspmins18;Database=Isolamento_Versao_2;uid=hspmApp;pwd=SoundG@rden=1")
cursor = cnxn.cursor()
try:
    for index, row in df_anti_join_excel.iterrows():
        if pd.isna(row['Resultado']):
      # Prepare the SQL query
            sql = """
            INSERT INTO [dbo].[CCIH]
               (
               [parametroDataInicial]
               ,[parametroDataFinal]
               ,[ParametroUsuarioLogado]
               ,[DataSistema]
               ,[Clinica]
               ,[NomeClinica]
               ,[NumeroPedido]
               ,[NomePaciente]
               ,[DataNascimento]
               ,[Nome]
               , [Resultado]
           
          
               ,[ComplementoResultado]
               ,[Ativo])
         VALUES
               (?,?,?,?,?,?,?,?,?,?,NULL,?,'A')
            """
            # Execute the query with parameters
            cursor.execute(sql, (row['parametroDataInicial'], row['parametroDataFinal'] ,row['ParametroUsuarioLogado'],row['DataSistema'], row['Clinica'] ,row['NomeClinica'],row['NumeroPedido'], row['NomePaciente'] ,row['DataNascimento'],row['Nome'],row['ComplementoResultado'] ))
         
        else:
            # Prepare the SQL query
            sql = """
            INSERT INTO [dbo].[CCIH]
               (
               [parametroDataInicial]
               ,[parametroDataFinal]
               ,[ParametroUsuarioLogado]
               ,[DataSistema]
               ,[Clinica]
               ,[NomeClinica]
               ,[NumeroPedido]
               ,[NomePaciente]
               ,[DataNascimento]
               ,[Nome]
               ,[Resultado]
           
          
               ,[ComplementoResultado]
               ,[Ativo])
         VALUES
               (?,?,?,?,?,?,?,?,?,?,?,?,'A')
            """
            # Execute the query with parameters
            cursor.execute(sql, (row['parametroDataInicial'], row['parametroDataFinal'] ,row['ParametroUsuarioLogado'],row['DataSistema'], row['Clinica'] ,row['NomeClinica'],row['NumeroPedido'], row['NomePaciente'] ,row['DataNascimento'],row['Nome'], row['Resultado'] ,row['ComplementoResultado'] ))
         
    # Commit all updates
    cnxn.commit()
    print("All updates were successful.")
except Exception.Error as e:
    print("Error occurred:", e)
    cnxn.rollback()  # Roll back in case of any error
finally:
    cursor.close()
    cnxn.close()

All updates were successful.


In [17]:

df_anti_join_excel

,parametroDataInicial,parametroDataFinal,ParametroUsuarioLogado,DataSistema,Clinica,NomeClinica,NumeroPedido,NomePaciente,DataNascimento,Nome,Resultado,ComplementoResultado,Ativo
0,2024-07-01,2024-08-19,HENRIQLUSTOSA,2024-07-29,2GER,GERIATRIA_2º AND - AMB,10823657,ARACY RHORMENS,1935-06-18,UROCULTURA,Isolado [1]: Klebsiella pneumoniae,"Cepa resistente a carbapenemicos, consultar CCIH.",NaN
4,2024-07-01,2024-08-19,HENRIQLUSTOSA,2024-08-03,1UT6,UTI_6º AND,10846896,CELINA PEREIRA CORDEIRO,1945-11-24,CULTURA GERAL,Isolado [1]: Acinetobacter baumannii complex,"Cepa resistente a carbapenemicos, consultar CCIH.",NaN
5,2024-07-01,2024-08-19,HENRIQLUSTOSA,2024-08-06,9C,9º AND_C ENFERMARIA GERAL,10847670,CELINA PEREIRA CORDEIRO,1945-11-24,UROCULTURA,Isolado [1]: Klebsiella pneumoniae,"Cepa resistente a carbapenemicos, consultar CCIH.",NaN
7,2024-07-01,2024-08-19,HENRIQLUSTOSA,2024-08-12,1PS,PRONTO SOCORRO ADULTO_1º AND,10839354,CLEA APARECIDA DE SOUZA,1954-04-09,CULTURA GERAL,Isolado [1]: Klebsiella pneumoniae,"Cepa resistente a carbapenemicos, consultar CCIH.",NaN
8,2024-07-01,2024-08-19,HENRIQLUSTOSA,2024-08-12,1PS,PRONTO SOCORRO ADULTO_1º AND,10839354,CLEA APARECIDA DE SOUZA,1954-04-09,CULTURA GERAL,Isolado [2]: Enterococcus faecium,Enterococcus resistente a glicopeptideos (VRE)...,NaN
10,2024-07-01,2024-08-19,HENRIQLUSTOSA,2024-07-31,10A,10º AND_ UTI,10839211,DIONYSIO CAETANO,1939-07-14,HEMOCULTURA 2º AMOSTRA,Isolado [1]: Pseudomonas aeruginosa,"Cepa resistente a carbapenemicos, consultar CCIH.",NaN
11,2024-07-01,2024-08-19,HENRIQLUSTOSA,2024-08-11,12A,12º AND_ - ENF,10849938,EDIVAR GRACIANO DA SILVA,1945-03-03,CULTURA GERAL,Isolado [1]: Enterococcus faecium,Enterococcus resistente a glicopeptideos (VRE)...,NaN
15,2024-07-01,2024-08-19,HENRIQLUSTOSA,2024-07-25,1PS,PRONTO SOCORRO ADULTO_1º AND,10844180,EVELINE APARECIDA DE ALMEIDA CARLOS,1983-10-12,CULTURA GERAL,Isolado [1]: Klebsiella pneumoniae,"Cepa resistente a carbapenemicos, consultar CCIH.",NaN
16,2024-07-01,2024-08-19,HENRIQLUSTOSA,2024-08-09,10A,10º AND_ UTI,10824488,LENI DA SILVA,1946-08-06,HEMOCULTURA 1º AMOSTRA,Isolado [1]: Acinetobacter baumannii,"Cepa resistente a carbapenemicos, consultar CCIH.",NaN
17,2024-07-01,2024-08-19,HENRIQLUSTOSA,2024-08-05,10A,10º AND_ UTI,10839219,LENI DA SILVA,1946-08-06,HEMOCULTURA 1º AMOSTRA,Isolado [2]: Acinetobacter baumannii,"Cepa resistente a carbapenemicos, consultar CCIH.",NaN


In [18]:
df_anti_join_sql

,posicao,parametroDataInicial_x,parametroDataFinal_x,ParametroUsuarioLogado_x,DataSistema_x,Clinica_x,NomeClinica_x,NumeroPedido,NomePaciente_x,DataNascimento_x,Nome_x,Resultado,Prontuario,id,ComplementoResultado_x,Ativo


In [19]:
df_resultados_inner_join

,parametroDataInicial_x,parametroDataFinal_x,ParametroUsuarioLogado_x,DataSistema_x,Clinica_x,NomeClinica_x,NumeroPedido,NomePaciente_x,DataNascimento_x,Nome_x,...,DataSistema_y,Clinica_y,NomeClinica_y,NomePaciente_y,DataNascimento_y,Nome_y,Prontuario,id,ComplementoResultado_y,Ativo
0,2024-07-01,2024-08-19,HENRIQLUSTOSA,2024-07-17,10A,10º AND_ UTI,10837803,ARANDI LIMA DA SILVA,1943-04-12,CULTURA GERAL,...,2024-07-17,10A,10º AND_ UTI,ARANDI LIMA DA SILVA,1943-04-12,CULTURA GERAL,4315131,4526,"Cepa resistente a carbapenemicos, consultar CCIH.",A
1,2024-07-01,2024-08-19,HENRIQLUSTOSA,2024-07-23,7CC,CENTRO CIRURGICO_7º AND,10829588,CECILIA REGINA DOS SANTOS DE OLIVEIRA,1948-09-12,CULTURA GERAL,...,2024-07-23,7CC,CENTRO CIRURGICO_7º AND,CECILIA REGINA DOS SANTOS DE OLIVEIRA,1948-09-12,CULTURA GERAL,1113992,4533,"Cepa resistente a carbapenemicos, consultar CCIH.",A
2,2024-07-01,2024-08-19,HENRIQLUSTOSA,2024-07-23,7CC,CENTRO CIRURGICO_7º AND,10829589,CECILIA REGINA DOS SANTOS DE OLIVEIRA,1948-09-12,CULTURA GERAL,...,2024-07-23,7CC,CENTRO CIRURGICO_7º AND,CECILIA REGINA DOS SANTOS DE OLIVEIRA,1948-09-12,CULTURA GERAL,1113992,4534,"Cepa resistente a carbapenemicos, consultar CCIH.",A
3,2024-07-01,2024-08-19,HENRIQLUSTOSA,2024-07-05,1PS2,PRONTO SOCORRO _2º AND,10797322,CLEA APARECIDA DE SOUZA,1954-04-09,CULTURA GERAL,...,2024-07-05,1PS2,PRONTO SOCORRO _2º AND,CLEA APARECIDA DE SOUZA,1954-04-09,CULTURA GERAL,2836294,4529,"Cepa resistente a carbapenemicos, consultar CCIH.",A
4,2024-07-01,2024-08-19,HENRIQLUSTOSA,2024-07-26,12A,12º AND_ - ENF,10841518,CREMILDA ROSA DE ANDRADE CORREA,1939-09-05,UROCULTURA,...,2024-07-26,12A,12º AND_ - ENF,CREMILDA ROSA DE ANDRADE CORREA,1939-09-05,UROCULTURA,1301647,4535,Enterococcus resistente a glicopeptideos (VRE)...,A
5,2024-07-01,2024-08-19,HENRIQLUSTOSA,2024-07-05,7CC,CENTRO CIRURGICO_7º AND,10829602,EDNA MARINI DE CAMPOS,1967-05-17,CULTURA GERAL,...,2024-07-05,7CC,CENTRO CIRURGICO_7º AND,EDNA MARINI CAMPOS,1967-05-17,CULTURA GERAL,8004806,4530,"Cepa resistente a carbapenemicos, consultar CCIH.",A
6,2024-07-01,2024-08-19,HENRIQLUSTOSA,2024-07-15,1PCH,CHOQUE,10822500,ERIVALDO PEDRO DA SILVA,1952-08-29,UROCULTURA,...,2024-07-15,1PCH,CHOQUE,ERIVALDO PEDRO DA SILVA,1952-08-29,UROCULTURA,1256593,4527,"Cepa resistente a carbapenemicos, consultar CCIH.",A
7,2024-07-01,2024-08-19,HENRIQLUSTOSA,2024-07-15,1PS,PRONTO SOCORRO ADULTO_1º AND,10831623,ERIVALDO PEDRO DA SILVA,1952-08-29,HEMOCULTURA 2º AMOSTRA,...,2024-07-15,1PS,PRONTO SOCORRO ADULTO_1º AND,ERIVALDO PEDRO DA SILVA,1952-08-29,HEMOCULTURA 2º AMOSTRA,1256593,4528,"Cepa resistente a carbapenemicos, consultar CCIH.",A
8,2024-07-01,2024-08-19,HENRIQLUSTOSA,2024-07-05,4,4ºAND- ONCO,10802551,LUCIANA PAULOS DE BARROS BLOISE,1974-01-22,UROCULTURA,...,2024-07-05,4,4ºAND- ONCO,LUCIANA PAULOS DE BARROS BLOISE,1974-01-22,UROCULTURA,5409701,4531,"Cepa resistente a carbapenemicos, consultar CCIH.",A
9,2024-07-01,2024-08-19,HENRIQLUSTOSA,2024-07-03,6URO,UROLOGIA_6º AND - AMB,10836356,MARIO SERGIO JUSTO,1954-03-13,UROCULTURA,...,2024-07-03,6URO,UROLOGIA_6º AND - AMB,MARIO SERGIO JUSTO,1954-03-13,UROCULTURA,10857738,4525,"Cepa resistente a carbapenemicos, consultar CCIH.",A


In [20]:
df_anti_join_excel

,parametroDataInicial,parametroDataFinal,ParametroUsuarioLogado,DataSistema,Clinica,NomeClinica,NumeroPedido,NomePaciente,DataNascimento,Nome,Resultado,ComplementoResultado,Ativo
0,2024-07-01,2024-08-19,HENRIQLUSTOSA,2024-07-29,2GER,GERIATRIA_2º AND - AMB,10823657,ARACY RHORMENS,1935-06-18,UROCULTURA,Isolado [1]: Klebsiella pneumoniae,"Cepa resistente a carbapenemicos, consultar CCIH.",NaN
4,2024-07-01,2024-08-19,HENRIQLUSTOSA,2024-08-03,1UT6,UTI_6º AND,10846896,CELINA PEREIRA CORDEIRO,1945-11-24,CULTURA GERAL,Isolado [1]: Acinetobacter baumannii complex,"Cepa resistente a carbapenemicos, consultar CCIH.",NaN
5,2024-07-01,2024-08-19,HENRIQLUSTOSA,2024-08-06,9C,9º AND_C ENFERMARIA GERAL,10847670,CELINA PEREIRA CORDEIRO,1945-11-24,UROCULTURA,Isolado [1]: Klebsiella pneumoniae,"Cepa resistente a carbapenemicos, consultar CCIH.",NaN
7,2024-07-01,2024-08-19,HENRIQLUSTOSA,2024-08-12,1PS,PRONTO SOCORRO ADULTO_1º AND,10839354,CLEA APARECIDA DE SOUZA,1954-04-09,CULTURA GERAL,Isolado [1]: Klebsiella pneumoniae,"Cepa resistente a carbapenemicos, consultar CCIH.",NaN
8,2024-07-01,2024-08-19,HENRIQLUSTOSA,2024-08-12,1PS,PRONTO SOCORRO ADULTO_1º AND,10839354,CLEA APARECIDA DE SOUZA,1954-04-09,CULTURA GERAL,Isolado [2]: Enterococcus faecium,Enterococcus resistente a glicopeptideos (VRE)...,NaN
10,2024-07-01,2024-08-19,HENRIQLUSTOSA,2024-07-31,10A,10º AND_ UTI,10839211,DIONYSIO CAETANO,1939-07-14,HEMOCULTURA 2º AMOSTRA,Isolado [1]: Pseudomonas aeruginosa,"Cepa resistente a carbapenemicos, consultar CCIH.",NaN
11,2024-07-01,2024-08-19,HENRIQLUSTOSA,2024-08-11,12A,12º AND_ - ENF,10849938,EDIVAR GRACIANO DA SILVA,1945-03-03,CULTURA GERAL,Isolado [1]: Enterococcus faecium,Enterococcus resistente a glicopeptideos (VRE)...,NaN
15,2024-07-01,2024-08-19,HENRIQLUSTOSA,2024-07-25,1PS,PRONTO SOCORRO ADULTO_1º AND,10844180,EVELINE APARECIDA DE ALMEIDA CARLOS,1983-10-12,CULTURA GERAL,Isolado [1]: Klebsiella pneumoniae,"Cepa resistente a carbapenemicos, consultar CCIH.",NaN
16,2024-07-01,2024-08-19,HENRIQLUSTOSA,2024-08-09,10A,10º AND_ UTI,10824488,LENI DA SILVA,1946-08-06,HEMOCULTURA 1º AMOSTRA,Isolado [1]: Acinetobacter baumannii,"Cepa resistente a carbapenemicos, consultar CCIH.",NaN
17,2024-07-01,2024-08-19,HENRIQLUSTOSA,2024-08-05,10A,10º AND_ UTI,10839219,LENI DA SILVA,1946-08-06,HEMOCULTURA 1º AMOSTRA,Isolado [2]: Acinetobacter baumannii,"Cepa resistente a carbapenemicos, consultar CCIH.",NaN


In [21]:
df_anti_join_excel.size


260